In [32]:
import torch
#from torch.autograd import Variable
#from torch import nn
#from torch.nn import functional as F
import numpy as np
import math
import dlc_practical_prologue as prologue

In [34]:
t=list(np.zeros(18+1))
print(t)
t[1]=np.array([5,4])
t[2]=7
t[1]
print(t)
#r = torch.tanh(b1)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, array([5, 4]), 7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [35]:
for i in range (4,0,-1):
    print(i)

4
3
2
1


In [36]:
class linear():
    def __init__(self,start_layer_size,end_layer_size):
        assert type(start_layer_size)== int 
        assert type(end_layer_size) == int
        if (start_layer_size > 0 and end_layer_size > 0):
            
            self.m = start_layer_size
            self.n = end_layer_size
            self.w = torch.Tensor(self.m,self.n).normal_()
            self.b = torch.Tensor(self.m,1).normal_()
        else :
            raise "error"
        
    def forward(self, x):
        #print('linear forward called')
        return torch.mm(self.w , x)+self.b
    
    def backward(self,x):
        print("backward_linear called")
        return x,self.w
    
        

In [37]:
class Relu():
     
    def forward(self,x):
        y = torch.zeros_like(x)
        return torch.max(x,y)
    
    def backward(self,x):
        u=x.clone()
        u[u>0]=1
        u[u<0]=0
        return u


In [38]:
class Tanh():
     
    def forward(self,x):
        return torch.tanh(x)
       
    
    def backward(self,x):
        return torch.ones_like(x)-torch.tanh(x)*torch.tanh(x)


In [39]:
class Identity():
    
    def forward(self,x):
        return x
    
    def backward(self,x):
        return torch.ones_like(x)

In [40]:
f=Identity()

In [41]:

p = torch.zeros(3,2)+2
print(p)


 2  2
 2  2
 2  2
[torch.FloatTensor of size 3x2]



In [42]:
der2 = f.forward(p)
print(der2)


 2  2
 2  2
 2  2
[torch.FloatTensor of size 3x2]



In [44]:
#f.forward(p)
#f.backward(p)

In [45]:

y = p.clone()

p[p>0]=1
p[p<0]=0
p


 1  1
 1  1
 1  1
[torch.FloatTensor of size 3x2]

In [46]:
# class function():
    
#     def __init__(self):
#         self.type1 = np.maximum
#         self.type2 = torch.tanh
#         #self.flag = flag
    

        
#     def Relu(self,a):
#         return self.type1(a,0)
        
    
#     def Tanh(self,b):
#         return  self.type2(b)
    
#     def Identity(self,c):
        
            
            

In [48]:
#type(np.maximum(a,0))

In [49]:
def luk(*a):
    print(type(a))
    for i in a:
        print (i)

In [50]:
luk(1,2,43)

<class 'tuple'>
1
2
43


In [52]:
#model = linear(2,4)
#F.Relu(model.b)

In [53]:
 def __structrize__(activations):
        que=[]
        l=len(activations)
        print(l)
        for i in range(l):
            que.append(activations[i])
            if(i<l-1):
                if(type(activations[i]).__name__=='linear' and type(activations[i+1]).__name__=='linear'):
                    que.append(Identity())
        
        if (type(activations[-1]).__name__=='linear'):
            que.append(Identity())
        
        return que

In [54]:
class sequential(): #list of operation
    def __init__(self,*operations):
        self.l = len(operations)
        self.result = []
        self.initial_operations = operations
        self.operations = __structrize__(self.initial_operations )
#         linear.__init__(self,in_node,out_node)
#         function.__init__(self)
        self.delta =[] # dl_ds
        self.forward_flag=False
#         self.w =[]
#         self.w.append(0)
#         self.w.append(weight)
#         self.b=bias
#         self.l = int(num_layers)
#         self.s = list(np.zeros(self.l + 1) ) # first element remain 0  each time
#         self.x = list(np.zeros(self.l + 1) )
#         #derivatives
#         self.dl_dx = list(np.zeros(self.l+1) ) # first element remain 0  each time
          
        self.dl_dw = []
        self.dl_db = [] 
        
    '''
    def structrize(self,activations):
        que=[]
        l=len(activations)
        print(l)
        for i in range(l):
            que.append(activations[i])
            if(i<l-1):
                if(type(activations[i]).__name__=='linear' and type(activations[i+1]).__name__=='linear'):
                    que.append(Identity())
        
        if (type(activations[-1]).__name__=='linear'):
            que.append(Identity())
        
        return que

    self.operations= structrize(self.initial_operations)
    '''
    def sigma(self , inp):
        return torch.tanh(inp)
    def dsigma(x):
        
        return 1 - torch.mul(torch.tanh(x),torch.tanh(x))
    
    def loss(self ,output,target):
        return torch.sum(torch.pow(output-target,2))
    
    
    def dloss(output,target): # dloss is derivative of loss
    #print("dloss_shape(v-t) = ",v.shape , "-",t.shape)
        return 2*(output-target)
    
    def forward(self,x):
        self.forward_flag=True
        self.result.append(x)
        for op in self.operations:
            self.result.append(op.forward(self.result[-1]))
            #print('sequential forward called ,result[i]:',self.result[-1])
        return self.result   
        
    def backward_pass(self,target):
        
        if(self.forward_flag==False):
            raise ValueError("forward hasn't been called.")
        self.delta.append(-2*(target-self.result[-1]))
        
        print("result",len(self.result))
        
        
        for i in range (self.l,0,-1): # calculating deltas
            #print("i = ",i)
            #print("delta length",len(self.delta))
            
            #print('delta = ',self.delta[-1])
            #print('current op= ',self.operations[i-1])
            self.delta.append(torch.mm(self.delta[-1],self.operations[i-1].backward(self.result[i-1])[1])) # [1] is to select w not x from linear_backward()
       
        self.delta.reverse()
        for i in range (self.l,0,-1): #calculating dl_dw's 
            print("i = ",i,"-------------------------------------------------------------------------")
            print("x_(l-1).shape" , self.result[i-1].shape)
            print("delta[",i,"]",self.delta[i].shape)
            #self.dl_dw.append(torch.mm(self.delta[i],torch.t(self.result[i-1])))
            self.dl_dw.append(torch.mm(self.result[i-1],self.delta[i]))
        
        self.dl_dw.reverse()
    
        self.dl_db=self.delta
        return self.delta, self.dl_dw, self.dl_db
            
            
#         self.dl_dx[self.l] = dloss(self.x[self.l] , target)
#         self.dl_ds[self.l] = torch.mul(dl_dx[self.l],dsigma(s[self.l]) )
        
#         torch.mul( dl_dx2 , dsigma(s2) )
        
#         for i in range(self.l-1,0,-1):  # it goes from l-1 to 1
        
#             #print("dl_dx2-->size = " , (dloss(x2,t)).shape)
#             #print('x2--->size' ,  x2.shape )
#             #print('x2 = ' ,x2)
#             dl_dx[i] = dloss(x[-1],t)
    

    
#             #print("dl_dx1-->size = " , (w2.t()).shape,' * ',dl_ds2.shape)
#             dl_dx[i] = torch.mm(w[i+1].t(),dl_ds[i+1])
    
#             #print("dl_ds1-->size = " , dl_dx1.shape,' .* ',dsigma(s1).shape)
#             dl_ds[i] = torch.mul(dl_dx[i] ,dsigma(s[i]) )
    
#             #print("dl_dw1-->size = " , dl_ds1.view(-1,1).shape,' * ',(x0.view(1,-1).shape))
                #self.result.append(op.forward(self.result[-1]))
            
    
    
    
    
#             dl_dw[i] = torch.mm(dl_ds1.view(-1,1),x0.view(1,-1))
#             dl_dw2 = dl_ds2.view(-1,1).mm(x1.view(1,-1))
    
    
#             dl_db1=dl_ds1;
#             dl_db2=dl_ds2;
    
#             #print(dl_dw1.shape,dl_dw2.shape,dl_db1.shape,dl_db2.shape)
#             #print("(dl_dw1,dl_dw2,dl_db1,dl_db2)")
#         return self.dl_dw,self.dl_db

In [55]:
obj1 = linear(5,2)
obj2 = linear(3,5)
obj3 = linear(1,3)

a = sequential(obj1,obj2,obj3)
x = torch.zeros(2,1)
q = a.forward(x)
#print(len(q))# تییتیتت
#print(q[1])
print("q",q)
b = a.backward_pass(0)
b

3
q [
 0
 0
[torch.FloatTensor of size 2x1]
, 
-1.1645
-0.3114
-1.0446
 0.3701
 0.2697
[torch.FloatTensor of size 5x1]
, 
-1.1645
-0.3114
-1.0446
 0.3701
 0.2697
[torch.FloatTensor of size 5x1]
, 
 0.7841
-1.0401
-2.0175
[torch.FloatTensor of size 3x1]
, 
 0.7841
-1.0401
-2.0175
[torch.FloatTensor of size 3x1]
, 
 2.0607
[torch.FloatTensor of size 1x1]
, 
 2.0607
[torch.FloatTensor of size 1x1]
]
result 7
backward_linear called


RuntimeError: size mismatch, m1: [1 x 1], m2: [3 x 5] at /opt/conda/conda-bld/pytorch_1503970438496/work/torch/lib/TH/generic/THTensorMath.c:1293

In [26]:
a.delta

[
 -3.1857
 [torch.FloatTensor of size 1x1]]

In [17]:
#torch.mm(obj3.w,a.delta[-1])
print(-2*(0-a.result[-1]))
print(torch.mm(a.delta[1],obj1.w))

NameError: name 'a' is not defined

In [18]:
a.dl_dw

NameError: name 'a' is not defined

In [25]:
# class sequential():
#     def __init__(self,weight=[],bias=[],num_layers=0):
#         self.w =[]
#         self.w.append(0)
#         self.w.append(weight)
#         self.b=bias
#         self.l = int(num_layers)
#         self.s = list(np.zeros(self.l + 1) ) # first element remain 0  each time
#         self.x = list(np.zeros(self.l + 1) )
#         #derivatives
#         self.dl_dx = list(np.zeros(self.l+1) ) # first element remain 0  each time
#         self.dl_ds = list(np.zeros(self.l+1) ) # first element remain 0  each time
#         self.dl_dw = list(np.zeros(self.l+1) ) # first element remain 0  each time
#         self.dl_db = list(np.zeros(self.l+1) ) # first element remain 0  each time
        
#     def sigma(self , inp):
#         return torch.tanh(inp)
#     def dsigma(x):
        
#         return 1 - torch.mul(torch.tanh(x),torch.tanh(x))
    
#     def loss(self ,output,target):
#         return torch.sum(torch.pow(output-target,2))
    
    
#     def dloss(output,target): # dloss is derivative of loss
#     #print("dloss_shape(v-t) = ",v.shape , "-",t.shape)
#         return 2*(output-target)
    
#     def forward(self, inp):
        
#         self.x[0] = inp
#         for i in range(1,self.l+1):
            
#             self.s[i] = torch.mm(self.w[i] , self.x[i-1])+self.b[i]
#             self.x[i] = self.sigma(self.s[i])
                
#         return self.s,self.x   
        
#     def backward_pass(self,target):

#         self.dl_dx[self.l] = dloss(self.x[self.l] , target)
#         self.dl_ds[self.l] = torch.mul(dl_dx[self.l],dsigma(s[self.l]) )
        
#         torch.mul( dl_dx2 , dsigma(s2) )
        
#         for i in range(self.l-1,0,-1):  # it goes from l-1 to 1
        
#             #print("dl_dx2-->size = " , (dloss(x2,t)).shape)
#             #print('x2--->size' ,  x2.shape )
#             #print('x2 = ' ,x2)
#             dl_dx[i] = dloss(x[-1],t)
    

    
#             #print("dl_dx1-->size = " , (w2.t()).shape,' * ',dl_ds2.shape)
#             dl_dx[i] = torch.mm(w[i+1].t(),dl_ds[i+1])
    
#             #print("dl_ds1-->size = " , dl_dx1.shape,' .* ',dsigma(s1).shape)
#             dl_ds[i] = torch.mul(dl_dx[i] ,dsigma(s[i]) )
    
#             #print("dl_dw1-->size = " , dl_ds1.view(-1,1).shape,' * ',(x0.view(1,-1).shape))
    
    
    
    
    
#             dl_dw[i] = torch.mm(dl_ds1.view(-1,1),x0.view(1,-1))
#             dl_dw2 = dl_ds2.view(-1,1).mm(x1.view(1,-1))
    
    
#             dl_db1=dl_ds1;
#             dl_db2=dl_ds2;
    
#             #print(dl_dw1.shape,dl_dw2.shape,dl_db1.shape,dl_db2.shape)
#             #print("(dl_dw1,dl_dw2,dl_db1,dl_db2)")
#         return self.dl_dw,self.dl_db
    
        
        
        
        
    
     
#        return self


# net = EEGNet()#.cuda(0)
# print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)))))
# criterion = nn.BCELoss()
# optimizer = optim.Adam(net.parameters())

In [425]:
model = sequential(linear(3,2),Tanh(),linear(5,3))

In [426]:
x = torch.Tensor(2,1).normal_()
x


-1.7598
-0.2378
[torch.FloatTensor of size 2x1]

In [427]:
model.forward(x)

[
 -1.7598
 -0.2378
 [torch.FloatTensor of size 2x1], 
  4.2575
 -2.9671
  1.6491
 [torch.FloatTensor of size 3x1], 
  0.9996
 -0.9947
  0.9287
 [torch.FloatTensor of size 3x1], 
 -0.0439
 -1.2658
 -1.1588
  2.5660
 -4.3117
 [torch.FloatTensor of size 5x1]]

In [29]:
weight = [w1,w2]
weight.append(w3)
print(weight)
weight[2]

[
 0  0
 0  0
 0  0
 0  0
[torch.FloatTensor of size 4x2]
, 
 0  0  0  0
[torch.FloatTensor of size 1x4]
, 
 1.1711  0.4335 -1.7343
-1.3360  0.8871  0.7680
 0.0571  0.2240  0.5520
[torch.FloatTensor of size 3x3]
]



 1.1711  0.4335 -1.7343
-1.3360  0.8871  0.7680
 0.0571  0.2240  0.5520
[torch.FloatTensor of size 3x3]

In [190]:
w1 =  torch.Tensor(3,4)
w2 = torch.Tensor(4,1)
torch.mm(w1,w2)
w1*w2

RuntimeError: inconsistent tensor size, expected r_ [3 x 4], t [3 x 4] and src [4 x 1] to have the same number of elements, but got 12, 12 and 4 elements respectively at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/TH/generic/THTensorMath.c:1036

In [30]:
def generate_disc_set(nb):
    input_data = torch.Tensor(nb, 2).uniform_(-0.5,0.5)
    target = input_data.pow(2).sum(1).sub(1 /(2* math.pi)).sign().mul(-1).add(1).div(2).long()
    return input_data, target

In [31]:
a,b=generate_disc_set(17)

In [32]:
print(a,b)


 0.3119 -0.3913
-0.1057 -0.2027
-0.0963 -0.0982
-0.4487 -0.4317
-0.0782  0.0065
-0.2271  0.1883
-0.4500 -0.0337
 0.4397 -0.2039
 0.4515  0.1811
-0.4512  0.3163
-0.0577 -0.2232
 0.3998 -0.4040
 0.0537 -0.1047
 0.3571  0.1396
 0.2403  0.1766
-0.1202 -0.1052
-0.4120  0.2709
[torch.FloatTensor of size 17x2]
 
 0
 1
 1
 0
 1
 1
 0
 0
 0
 0
 1
 0
 1
 1
 1
 1
 0
[torch.LongTensor of size 17]



In [502]:
luk

[7, 3, 1]

In [503]:
ops = [linear(4,2),linear(7,4),Relu(),linear(9,7),linear(1,9)]
print(len(ops))

5


In [504]:
# x = ops
# que=[]
# l=len(ops)
# for i in range(l):
#     que.append(ops[i])
#     if(i<l-1):
#         if(type(ops[i]).__name__=='linear' and type(ops[i+1]).__name__=='linear'):
#             que.append("identity")
        
# if (type(ops[-1]).__name__=='linear'):
#     que.append("identity")

In [505]:
# que

In [506]:
def structrize(operations):
    que=[]
    l=len(operations)
    print(l)
    for i in range(l):
        que.append(operations[i])
        if(i<l-1):
            if(type(operations[i]).__name__=='linear' and type(operations[i+1]).__name__=='linear'):
                que.append(Identity())
        
    if (type(operations[-1]).__name__=='linear'):
        que.append(Identity)
        
    return que


In [507]:
a = structrize(ops)

5


In [508]:
print(a)

[<__main__.linear object at 0x7f42e0771278>, <__main__.Identity object at 0x7f42e06ea438>, <__main__.linear object at 0x7f42e0771320>, <__main__.Relu object at 0x7f42e0771978>, <__main__.linear object at 0x7f42e0771048>, <__main__.Identity object at 0x7f42e06ea4e0>, <__main__.linear object at 0x7f42e0771898>, <class '__main__.Identity'>]
